# IBM HR Analytics Employee Attrition & Performance

### Step-by-Step Procedure

I.	Exploratory data analysis
    
    I.1. General exploration

    I.2. Numerical features
        I.2.1. Explore and clean Numerical features
        I.2.2. Missing data of Numerical features

    I.3. Categorical features
        I.3.1. Explore and clean Categorical features
        I.3.2. Missing data of Categorical features
        I.3.3. Transform Categorical features into Binary features (get_dummies)

    I.4. Merge numerical and binary features into one data set

II.	Feature engineering


III.	Modeling

    III.1. Models and metrics selection

    III.2. Hyperparameters tuning and model optimization
        III.2.1. Logistic regression
        III.2.2. DecisionTree Classifier
        III.2.3. XGBoost Classifier
        III.2.4. RandomForest Classifier

    III.3. Choosing the best model

IV. Prediction

# I. Exploratory Data Analysis

## I.1. General Exploration / Data Inspection

### 1.1. Importing necessary packages

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import warnings 
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", 80)
pd.set_option("display.max_rows", 80)

### 1.2. Loading the data sets

In [2]:
data = pd.read_csv("../input/ibm-hr-analytics-attrition-dataset/WA_Fn-UseC_-HR-Employee-Attrition.csv")
print(data.shape)
data.head()

### 1.3. Inspecting the data

In [3]:
# Checking if missing values
data.info()

In [4]:
# Checking Data distribution.
data.describe().T

In [5]:
# Checking number of unique values in each columns
count = 1
for x in data:
    print(f'{count}. {x}: {data[x].nunique()}')
    print(f'{data[x].value_counts()}', end = '\n----------\n\n' )    
    count += 1

Things to be noted from above result:
1. There is imbalance in the data. (In Y-variable, one of the class have very high number than the other.)
2. EmployeeCount, Over18 and StandardHours are constant variable(All the values in the columns are same) and needed to be dropped.
3. EmployeeNumber is a unique variable(All the values in the columns are completely different/ Primary key) and needed to be dropped.

In [6]:
# Dropping unnecessary columns.
data.drop(['EmployeeCount', 'Over18', 'StandardHours', 'EmployeeNumber'], axis = 1, inplace = True)

In [7]:
data.describe().T

## I.2. Numerical Features / Continuous variable

### 1.2.1. Exploring and Cleaning the continuous features

#### 1.2.1.1. Extracting Numerical features

In [8]:
cont_data = data.select_dtypes(exclude = ['object'] )
cont_data

#### 1.2.1.2. Data distribution

In [9]:
cont_data.hist(figsize = (25, 30), bins = 50, xlabelsize = 8, ylabelsize = 8)
plt.show()

#### 1.2.1.3. BarPlot

In [10]:
for i in cont_data:
    sns.barplot(y = cont_data[i], x = data['Attrition'])
    plt.show()

#### 1.2.1.4. Checking for outliers

In [11]:
# Using box plot for checking the presence of outliers.
for i in cont_data:
    plt.boxplot(cont_data[i], labels = [i], patch_artist=True)
    plt.show()

## patch_artist will show colour inside boxplot. by default it is false so no colour is shown.

## Some variables may show outliers here in the boxplot but actually have a meaningful reason for their presence.
## So, we will treat outliers only for necessary variable based on our domain understanding.

In [12]:
a = ['MonthlyIncome','NumCompaniesWorked', 'TotalWorkingYears', 'YearsInCurrentRole', 
     'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

In [13]:
type(a)

In [14]:
for i in a:
    sns.histplot(cont_data[i], kde = True, bins = 50, label = cont_data[i].skew())
    plt.legend(loc = 'upper right')
    plt.show()

### Skewness is asymmetry in a statistical distribution, in which the curve appears distorted or skewed either to the left or to the right.
A skewness value greater than 1 or less than -1 indicates a highly skewed distribution. A value between 0.5 and 1 or -0.5 and -1 is moderately skewed. A value between -0.5 and 0.5 indicates that the distribution is fairly symmetrical.

### .skew() function return unbiased skew over requested axis Normalized by N-1
### A legend is an area describing the elements of the graph.

In [15]:
out_vars = ['MonthlyIncome', 'TotalWorkingYears', 'TrainingTimesLastYear', 
            'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsInCurrentRole', 'YearsWithCurrManager']

### now i created a function for the treatment of outliers--

In [16]:
def outlierTreat(x):
    upper = x.quantile(.75) + 1.5 * (x.quantile(.75) - x.quantile(.25)) 
    lower = x.quantile(.25) - 1.5 * (x.quantile(.75) - x.quantile(.25))
    return x.clip(lower, upper)

#### .clip() is used to trim values at specified input threshold. We can use this function to put a lower limit and upper limit on the values that any cell can have in the dataframe

In [17]:
cont_data.loc[:, out_vars] = cont_data.loc[:, out_vars].apply(outlierTreat)
cont_data.loc[:, out_vars] 

In [18]:
# Using box plot for checking the presence of outliers.
for i in cont_data:
    plt.boxplot(cont_data[i], labels = [i])
    plt.show()

### 1.2.2. Missing value treatment

#### 1.2.2.1. Checking presence of missing values

In [19]:
for i in cont_data:
    print(f'{i}: {cont_data.shape[0] - cont_data[i].count()}')

#### The shape attribute for numpy arrays returns the dimensions of the array. If Y has n rows and m columns, then Y.shape is (n,m) . So Y.shape[0] is n .

**Hence, there are no missing values for Continuous variables.
Now, we plot a heatmap to visualize for multi-collinearity. However, we will be using other statistical method to remove multi-collinearity.******

In [20]:
# Finding the correlation.
corr = cont_data.corr()

# Setting the size of figure.
plt.rcParams['figure.figsize'] = (25, 25)

# Argument Trimming out the values above the main diagonal.
mask = np.triu(corr)

# Setting low correlation value to 0.
corr[(corr.values < 0.3) & (corr.values > -0.3)] = 0

# Plotting the heatmap.
sns.heatmap(corr, annot = True, fmt = '.2f', mask = mask)

### A correlation heatmap is a rectangular representation of data and it repeats the same data description twice because the categories are repeated on both axis for computing analysis. Hence, the same result is obtained twice. A correlation heatmap that presents data only once without repetition that is categories are correlated only once is known as a triangle correlation heatmap. 

### Since data is symmetric across the diagonal from left-top to right bottom the idea of obtaining a triangle correlation heatmap is to remove data above it so that it is depicted only once. The elements on the diagonal are the parts where categories of the same type correlate.

#### We will be keeping notes of these collinear variables.
#### Later after combining the categorical variables, we'll be dropping out multi-collinearity.

---

## I.3. Categorical Featues

### 1.3.1. Exploring and Cleaning the Categorical features

#### 1.3.1.1. Extracting Categorical features

In [21]:
cat_vars = data.select_dtypes(include = ['object'])
cat_vars

In [22]:
# Looking at the data distribution for different values.
plt.rcParams['figure.figsize'] = (6, 4)
for i in cat_vars:
    sns.countplot(x = cat_vars[i],palette="rainbow")
    plt.show()

In [23]:
# Count values of different values for each variables.
for i in cat_vars:
    print(cat_vars[i].value_counts(), end = '\n---------\n\n')

In [24]:
# The values in the features contains some special characters which are being replaced by '_'(underscore).

In [25]:
cat_vars.BusinessTravel = np.where(cat_vars.BusinessTravel == 'Non-Travel', 'Non_Travel', cat_vars.BusinessTravel)

cat_vars.Department = np.where(cat_vars.Department == 'Research & Development',
                               'Research_and_Development', cat_vars.Department)

In [26]:
def func(var):
    m = list()
    for i in var:
        x = i.split(" ")
        if len(x) > 1:
            m.append('_'.join(x))
        else:
            m.append(i)
    return m

#### split() method in Python split a string into a list of strings after breaking the given string by the specified separator. gfg

In [27]:
cat_vars = cat_vars.apply(func)
cat_vars

In [28]:
# Count values of different values for each variables.
for i in cat_vars:
    print(cat_vars[i].value_counts(), end = '\n---------\n\n')

### 1.3.2. Handling Missing values.

#### 1.3.2.1. Looking for presense of missing values.

In [29]:
cat_vars.info()

### There are no null values. So no need to worry about it. If null values were present, use mode imputation.

### 1.3.3. Transforming categorical variables.

#### 1.3.3.1. Creating Dummies for Categorical variables.

In [30]:
cat_data = cat_vars.copy()

In [31]:
cat_data = pd.get_dummies(cat_data, drop_first = True)
# cat_data.drop(['Attrition_No'], axis = 1, inplace = True)

In [32]:
# Finding the correlation.
corr = cat_data.corr()

# Setting the size of figure.
plt.rcParams['figure.figsize'] = (25, 25)

# Argument Trimming out the values above the main diagonal.
mask = np.triu(corr)

# Setting low correlation value to 0.
corr[(corr.values < 0.3) & (corr.values > -0.3)] = 0

# Plotting the heatmap.
sns.heatmap(corr, annot = True, fmt = '.2f', mask = mask)

### I.4. Merging numerical and categorical variables.

In [33]:
# Combining Numerical and Categorical data.
final_data = pd.concat([cont_data, cat_data], axis = 1)

In [34]:
final_data.head()

In [35]:
# Finding the correlation.
corr = final_data.corr()

# Setting the size of figure.
plt.rcParams['figure.figsize'] = (25, 25)

# Argument Trimming out the values above the main diagonal.
mask = np.triu(corr)

# Setting low correlation value to 0.
corr[(corr.values < 0.3) & (corr.values > -0.3)] = 0

# Plotting the heatmap.
sns.heatmap(corr, annot = True, fmt = '.2f', mask = mask)

# Feature Engineering and Feature Selection (Finding and dropping multi-collinear varibles)

### For classification based problems, we can either use VIF or Somers'D for finding the important varibles for the model.
### VIF helps to decrease multi-collinearity, whereas Somers'D helps to find the variable that increases the predictive power of my model.

In [36]:
# Importing packages for discovering muti-collinear features
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices

#### What is Patsy in Python?
#### patsy is a Python package for describing statistical models (especially linear models, or models that have a linear component) and building design matrices.

# VIF

In [37]:
# Separating X features from dataset and creating a model-parameter for statistical model building.
feature_columns = final_data.columns.difference(['Attrition_Yes'])
model_params = 'Attrition_Yes ~ ' + ' + '.join(feature_columns)
model_params

In [38]:
y, X = dmatrices(model_params, final_data, return_type = 'dataframe')

#### patsy.dmatrix    Construct a single design matrix given a formula_like and data.
#### patsy.dmatrices(formula_like, data, return_type)  Construct two design matrices given a formula_like and data.
#### This function is identical to dmatrix(), except that it requires (and returns) two matrices instead of one. By convention, the first matrix is the “outcome” or “y” data, and the second is the “predictor” or “x” data.




### What is design matrix in machine learning?
### Design matrix: A collection of feature vectors for different data points constitutes a design matrix. Each row of the matrix is one data point (i.e., one feature vector), and each column represents the values of a given feature across all of the data points

In [39]:
# Finding the VIF values and creating a dataframe to store these values corresponding to features name.
mul = pd.DataFrame()
mul['Features'] = X.columns

mul['VIF_values'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
mul

In [40]:
# Finding the variables based on the cut-off value for VIF.
# Theoretically, the value should be more than 5 but practically it is taken as more than 4.
# However, any value (4 or 5) is correct based on the business problems.

f_data = mul[mul.VIF_values > 5].reset_index(drop = True)
f_data

In [41]:
# Finding the correlation.
corr = final_data[f_data.Features[1:]].corr()

# Setting the size of figure.
plt.rcParams['figure.figsize'] = (25, 25)

# Argument Trimming out the values above the main diagonal.
mask = np.triu(corr)

# Setting low correlation value to 0.
corr[(corr.values < 0.3) & (corr.values > -0.3)] = 0

# Plotting the heatmap.
sns.heatmap(corr, annot = True, fmt = '.2f', mask = mask)

### We can either use VIF or Somers Delta method for feature engineering and feature selection.
### In vif we have to remove the columns having vif above 5(cut off value) and so we drop f_data from our final data and then use that data for modelling.

# Somers'D score

#### refer https://www.statisticshowto.com/somers-d/

In [42]:
# Separating X features from dataset and creating a model-parameter for statistical model building.
feature_columns = final_data.columns.difference(['Attrition_Yes'])
feature_columns

In [43]:
# Finding out the Somers'D value foreach variables.
col = list()
score = list()
for i in feature_columns:
    model_params = f'Attrition_Yes ~ {i}'
    log_reg = smf.logit(model_params, final_data).fit()
    somersD = 2 * metrics.roc_auc_score(final_data['Attrition_Yes'], log_reg.predict(final_data)) - 1   
    col.append(i)
    score.append(somersD)

In [44]:
# Making a dataframe for Somers'D score for different variables.
som = {'Column_name' : col,
        'SomersD_value' : score}
f_vars = pd.DataFrame(som)
f_vars

In [45]:
# Taking the cut-off value for Somers'D as 0.1
f_vars1 = f_vars[f_vars.SomersD_value >= 0.1]

In [46]:
print(len(f_vars1.Column_name))
f_vars1.Column_name

In [47]:
# Taking the cut-off value for Somers'D as 0.2
f_vars2 = f_vars[f_vars.SomersD_value >= 0.2]

In [48]:
print(len(f_vars2.Column_name))
f_vars2.Column_name

Ideally the cut-off value for Somers'D score should be greater than 0.2 but when we use this cut-off value, thw number of variables become very less. Thus, we use 0.1 kepping in mind that this may result in some prediction error.

# Modelling

#### from    sklearn.model_selection    import train_test_split, GridSearchCV
#### from    sklearn.feature_selection  import RFE
#### from    sklearn.linear_model       import LogisticRegression
#### from    sklearn                    import metrics
#### import  statsmodels.formula.api    as smf

#### (Already done above in importing necessary packages)

In [49]:
Model = list()
Accuracy = list()
AUC_score = list()

## Statistical Approach Logistic Regression.

In [50]:
# Creating a model-parameter for statistical model building.

model_params = 'Attrition_Yes ~ ' + ' + '.join(f_vars1.Column_name)
model_params

In [51]:
# Train test split for statistical models.
train, test = train_test_split(final_data, random_state = 42)

In [52]:
# model_params = '''Attrition_Yes ~ Age + BusinessTravel_Travel_Frequently + Department_Sales + DistanceFromHome + EnvironmentSatisfaction + JobInvolvement + JobSatisfaction + MaritalStatus_Single + MonthlyIncome + OverTime_Yes + StockOptionLevel + YearsWithCurrManager'''
l = smf.logit(model_params, train).fit()
print(l.summary2())

In [53]:
# Find the AUC score of the model.
Model.append('Stats_Log_Reg')
auc = metrics.roc_auc_score(test.Attrition_Yes, l.predict(test))
AUC_score.append(auc)
auc

In [54]:
# Find cut-off value for prediction using Response rate for now.
cutOff = train.Attrition_Yes.mean()
print('Cut-off value: ', cutOff)

# Find the Accuracy score of the model.
accuracy = metrics.accuracy_score(test.Attrition_Yes, l.predict(test) > cutOff)
Accuracy.append(accuracy)
accuracy

## ML Models

In [55]:
# Separating dependent and independent variables from final_data on the basis of Somers'D score
X = final_data[f_vars1.Column_name]
y = final_data['Attrition_Yes']

In [56]:
X.shape, y.shape

In [57]:
# Train-Test split for building ML models.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 12345)

In [58]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

### 1. ML based Logistic Regression

In [59]:
Model.append('ML_Log_reg')
log_reg = LogisticRegression(max_iter = 5000)
log_reg.fit(X_train, y_train)

In [60]:
# Find the Accuracy score of the model.
accuracy = metrics.accuracy_score(y_test, log_reg.predict(X_test))
Accuracy.append(accuracy)
accuracy

In [61]:
# Find the AUC score of the model.
auc = metrics.roc_auc_score(y_test, log_reg.predict(X_test))
AUC_score.append(auc)
auc

### 2. Decision Tree

In [62]:
from sklearn.tree import DecisionTreeClassifier

In [63]:
# Using GridSearchCv to cross-validate the model and find the value of hyper-parameters.
param_grid = {'max_depth' : range(2, 15)}

tree_clf = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = 10, n_jobs = -1, verbose = 1)
tree_clf.fit(X_train, y_train)

In [64]:
tree_clf.best_params_

In [65]:
# Creating the model using best estimator after CV.
Model.append('DTree')
tree_clf = tree_clf.best_estimator_

In [66]:
# Find the Accuracy score of the model.
accuracy = metrics.accuracy_score(y_test, tree_clf.predict(X_test))
Accuracy.append(accuracy)
accuracy

In [67]:
# Find the AUC score of the model.
auc = metrics.roc_auc_score(y_test, tree_clf.predict(X_test))
AUC_score.append(auc)
auc

### 3. Random Forest

In [68]:
from sklearn import ensemble as en

In [69]:
# Using GridSearchCv to cross-validate the model and find the value of hyper-parameters.

param_grid = {'n_estimators' : [20, 30, 40, 50, 60, 70, 80, 90, 100], 
                 'max_features' : range(2, 15)}

rf_clf = en.RandomForestClassifier()
rf_clf = GridSearchCV(rf_clf, param_grid, cv = 5, n_jobs = -1, scoring = 'roc_auc', verbose = 1)
rf_clf.fit(X_train, y_train)

In [70]:
rf_clf.best_params_

In [71]:
# Creating the model using best estimator after CV.
Model.append('RForest')
rf_clf = rf_clf.best_estimator_

In [72]:
# Find the Accuracy score of the model.
accuracy = metrics.accuracy_score(y_test, rf_clf.predict(X_test))
Accuracy.append(accuracy)
accuracy

In [73]:
# Find the AUC score of the model.
auc = metrics.roc_auc_score(y_test, rf_clf.predict(X_test))
AUC_score.append(auc)
auc

### 4. Gradient Boosting

In [74]:
# Using GridSearchCv to cross-validate the model and find the value of hyper-parameters.

param_grid = {'n_estimators' : [40, 50, 60, 70, 80, 90, 100, 110, 120, 130], 
#               'learning_rate' : [10 ** x for x in range(-3, 2)],
                 'max_features' : range(2, 15)}

gb_clf = en.GradientBoostingClassifier()
gb_clf = GridSearchCV(gb_clf, param_grid, cv = 5, n_jobs = -1, scoring = 'roc_auc', verbose = 1)
gb_clf.fit(X_train, y_train)

In [75]:
gb_clf.best_params_

In [76]:
# Creating the model using best estimator after CV.
Model.append('GBoost')
gb_clf = gb_clf.best_estimator_

In [77]:
# Find the Accuracy score of the model.
accuracy = metrics.accuracy_score(y_test, gb_clf.predict(X_test))
Accuracy.append(accuracy)
accuracy

In [78]:
# Find the AUC score of the model.
auc = metrics.roc_auc_score(y_test, gb_clf.predict(X_test))
AUC_score.append(auc)
auc

### 5. Xtreme Gradient Boosting

In [79]:
from xgboost import XGBRFClassifier

In [80]:
# # Using GridSearchCv to cross-validate the model and find the value of hyper-parameters.

# param_grid = {'n_estimators' : [20, 30, 40, 50, 60, 70, 80], 
# #               'learning_rate' : [10 ** x for x in range(-3, 2)],
#                  'max_features' : range(2, 15)}

# xgb_clf = XGBRFClassifier(use_label_encoder=False, objective='reg:squarederror')
# xgb_clf = GridSearchCV(xgb_clf, param_grid, cv = 5, n_jobs = -1, scoring = 'roc_auc', verbose = 1)
# xgb_clf.fit(X_train, y_train)

In [81]:
# xgb_clf.best_params_

In [82]:
# # Creating the model using best estimator after CV.
# Model.append('XGBoost')
# xgb_clf = xgb_clf.best_estimator_

In [83]:
# Cross-validating wasn't giving any better output than without CV. Hence, CV was not used(done).

In [84]:
Model.append('XGBoost')
xgb_clf = XGBRFClassifier(use_label_encoder=False, objective='reg:squarederror')
xgb_clf.fit(X_train, y_train)

In [85]:
# Find the Accuracy score of the model.
accuracy = metrics.accuracy_score(y_test, xgb_clf.predict(X_test))
Accuracy.append(accuracy)
accuracy

In [86]:
# Find the AUC score of the model.
auc = metrics.roc_auc_score(y_test, xgb_clf.predict(X_test))
AUC_score.append(auc)
auc

In [87]:
# Looking at outcoems for all different models.
comp = {
    'Model' : Model,
    'AUC_score' : AUC_score,
    'Accuracy' : Accuracy
}
pd.DataFrame(comp)

# Have A Great Day.

# EOF

# -----Coming Soon-----

1. Use of RFE for feature elimination.
2. Handling Imbalance data Using RandomSampling, SMOTE and SMOTETomek approaches

In [88]:
# from sklearn.tree import DecisionTreeClassifier

In [89]:
# rfe = RFE(DecisionTreeClassifier(random_state = 1234), n_features_to_select = 15)
# rfe.fit(X, y)

In [90]:
# features = X.columns[rfe.support_]
# X = X[features]
# features